In [1]:
import pandas as pd
import numpy as np
from itertools import combinations 
from itertools import permutations 

In [1]:
# Generates random rankings of num_candidates number of candidates.
# equal_pref=True means a ranking allows two candidates to be equal
# in preference.
def generate_random_ranking(num_candidates, num_rankings, equal_pref=True):
    if not equal_pref:
        return np.asarray([np.random.permutation(np.arange(num_candidates) + 1) for _ in range(num_rankings)])
    else:
        return np.random.choice(np.arange(num_candidates) + 1, (num_rankings, num_candidates))
    

# Creates the voter preference table given all the rankings, rankings
# are expected to be an array, where each row contains the rankings, 
# the first element is the ranking for the 1st candidate, 2nd column is
# for 2nd candidate, etc
def generate_preference_table(rankings):
    df = pd.DataFrame(rankings, columns = np.arange(rankings.shape[1]) + 1)
    pref_df = pd.DataFrame(index = permutations(np.arange(rankings.shape[1]) + 1, 2), dtype=int)
    for elem in permutations([1, 2, 3, 4], 2):
        pref_df.loc[elem,"A>B"] = int(df[df[elem[0]] < df[elem[1]]].shape[0])
        pref_df.loc[elem,"A=B"] = int(df[df[elem[0]] == df[elem[1]]].shape[0])
    return pref_df.fillna(0).astype(int)

# Scores all orderings and returns a dataframe sorted by order.
# Scores are computed by summing matching rows in preference table, 
# ex. ranking 1, 2, 3, 4 would pull the A>B for (1, 2), (1, 3), (1, 4)
# (2, 3), (2, 4) and (3, 4)
def Score(pref_df, num_candidates):
    scores = {}
    for elem in permutations(np.arange(num_candidates) + 1):
        rows = list(combinations(elem, 2))
        scores[elem] = pref_df.loc[rows,"A>B"].sum()
    scores = pd.DataFrame(index = scores.keys(), data = scores.values(), columns = ['score'])
    return scores.sort_values(by='score', ascending=False)

In [155]:
rankings = generate_random_ranking(4, 1000)
pref_table = generate_preference_table(rankings)
Scores = Score(pref_table, 4)
Scores.head()

,score
"(1, 3, 4, 2)",2298
"(1, 3, 2, 4)",2293
"(1, 4, 3, 2)",2288
"(3, 1, 4, 2)",2282
"(3, 1, 2, 4)",2277


# Testing that preference table sums to num_rankings

In [153]:
# All of these should be equal to num_rankings. Our implementation of the
# preference table is slightly different. So for (1, 2) we sum columns 
# 'A>B' and 'A=B' for (1, 2) and just column 'A>B' for row (2, 1)
num_rankings = 10000
num_trials = 1000

for _ in range(num_trials):
    rankings = generate_random_ranking(4, num_rankings)
    pref_table = generate_preference_table(rankings)

    for elem in permutations([1, 2, 3, 4], 2):
        num = pref_table.loc[[elem],:].sum(axis=1).values[0]
        num = num + pref_table.loc[[(elem[1], elem[0])], 'A>B'].values[0]
        if num != num_rankings:
            print("Failed test, " + elem + " did not sum to " + num_rankings + ".")
print('Success')

Success


# Testing with Wikipedia Example

https://en.wikipedia.org/wiki/Kemeny%E2%80%93Young_method

In [12]:
Key = {
    "Memphis" : 1,
    "Nashville" : 2,
    "Chattanooga" : 3,
    "Knoxville" : 4
}

In [138]:
Memphis_votes = np.tile([1, 2, 3, 4], (42, 1))
Nashville_votes = np.tile([4, 1, 2, 3], (26, 1))
Chattanooga_votes = np.tile([4, 3, 1, 2], (15, 1))
Knoxville_votes = np.tile([4, 3, 2, 1], (17, 1))

Tennessee_rankings = np.concatenate([Memphis_votes, Nashville_votes, Chattanooga_votes, Knoxville_votes])

## Manually entering correct outcomes from Wikipedia

In [180]:
# Manually enter preference data from Wikipedia to compare to our generated version
wiki_pref_df = pd.DataFrame(index = permutations(np.arange(4) + 1, 2), columns = ['A>B', 'A=B'])
wiki_pref_df.loc[(1, 2), "A>B"] = 42
wiki_pref_df.loc[(1, 3), "A>B"] = 42
wiki_pref_df.loc[(1, 4), "A>B"] = 42
wiki_pref_df.loc[(2, 3), "A>B"] = 68
wiki_pref_df.loc[(2, 4), "A>B"] = 68
wiki_pref_df.loc[(3, 4), "A>B"] = 83
wiki_pref_df.loc[(2, 1), "A>B"] = 58
wiki_pref_df.loc[(3, 1), "A>B"] = 58
wiki_pref_df.loc[(4, 1), "A>B"] = 58
wiki_pref_df.loc[(3, 2), "A>B"] = 32
wiki_pref_df.loc[(4, 2), "A>B"] = 32
wiki_pref_df.loc[(4, 3), "A>B"] = 17
wiki_pref_df.loc[:,'A=B'] = 0

wiki_pref_df = wiki_pref_df.astype(int)

# Manually enter ranking scores to compare to ours

wiki_scores = pd.DataFrame(index = permutations([1, 2, 3, 4]), columns = ['score'])
wiki_scores.loc[[(1, 2, 3, 4)],'score'] = 345
wiki_scores.loc[[(1, 2, 4, 3)],'score'] = 279
wiki_scores.loc[[(1, 3, 2, 4)],'score'] = 309
wiki_scores.loc[[(1, 3, 4, 2)],'score'] = 273
wiki_scores.loc[[(1, 4, 2, 3)],'score'] = 243
wiki_scores.loc[[(1, 4, 3, 2)],'score'] = 207

wiki_scores.loc[[(2, 1, 3, 4)],'score'] = 361
wiki_scores.loc[[(2, 1, 4, 3)],'score'] = 295
wiki_scores.loc[[(2, 3, 1, 4)],'score'] = 377
wiki_scores.loc[[(2, 3, 4, 1)],'score'] = 393
wiki_scores.loc[[(2, 4, 1, 3)],'score'] = 311
wiki_scores.loc[[(2, 4, 3, 1)],'score'] = 327

wiki_scores.loc[[(3, 1, 2, 4)],'score'] = 325
wiki_scores.loc[[(3, 1, 4, 2)],'score'] = 289
wiki_scores.loc[[(3, 2, 1, 4)],'score'] = 341
wiki_scores.loc[[(3, 2, 4, 1)],'score'] = 357
wiki_scores.loc[[(3, 4, 1, 2)],'score'] = 305
wiki_scores.loc[[(3, 4, 2, 1)],'score'] = 321

wiki_scores.loc[[(4, 1, 2, 3)],'score'] = 259
wiki_scores.loc[[(4, 1, 3, 2)],'score'] = 223
wiki_scores.loc[[(4, 2, 1, 3)],'score'] = 275
wiki_scores.loc[[(4, 2, 3, 1)],'score'] = 291
wiki_scores.loc[[(4, 3, 1, 2)],'score'] = 239
wiki_scores.loc[[(4, 3, 2, 1)],'score'] = 255

wiki_scores = wiki_scores.astype(int)

## Compare tables

In [204]:
Tennessee_pref_table = generate_preference_table(Tennessee_rankings)
print("Checking if preference tables match: ", Tennessee_pref_table.equals(wiki_pref_df))

Tennessee_scores = Score(Tennessee_pref_table, 4)
val = np.all([Tennessee_scores.loc[[elem],'score'].values[0] == wiki_scores.loc[[elem], 'score'].values[0]
            for elem in permutations([1, 2, 3, 4])])
print("Checking if Score tables match: ", val)


('Checking if preference tables match: ', True)
('Checking if Score tables match: ', True)


Ayyyyyyyyyy

# Connect it to a google sheet that gets data from a google form.